In [2]:
!pip   install xlsxwriter
!pip install python-pptx

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
import os
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_ALIGN
from pptx.dml.color import RGBColor
from pptx.enum.shapes import MSO_SHAPE

# 1. Conexión con Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


## **Danone**

In [8]:

# --- 1. CONFIGURACIÓN ---
RUTA_ENTRADA = '/content/drive/MyDrive/Danone_SEO/Danone/Danone-data-enero.csv'
DIRECTORIO_SALIDA = '/content/drive/MyDrive/Danone_SEO/Danone/'
TEMPLATE_PATH = '/content/drive/MyDrive/Danone 2026/7. Holistic /Assets/SEO_Danone-Holistic-template.pptx'

# Colores
DANONE_BLUE = RGBColor(0, 77, 165)
DANONE_WHITE = RGBColor(255, 255, 255)

# --- 2. FUNCIONES DE APOYO PPT ---

def limpiar_area_grafico(slide):
    """Intenta limpiar el centro de la slide para poner el gráfico nuevo"""
    for shape in list(slide.shapes):
        # Si es un placeholder de imagen o un cuadro vacío grande, lo borramos
        if shape.is_placeholder and shape.placeholder_format.type == 18: # PICTURE
            sp = shape._element
            sp.getparent().remove(sp)

def poner_grafico(slide, ruta_img):
    """Inserta la imagen del gráfico centrada"""
    # Ajuste manual para que quede bien en el template Danone
    left = Inches(1)
    top = Inches(2.2)
    height = Inches(4.2)
    slide.shapes.add_picture(ruta_img, left, top, height=height)

# --- 3. PROCESO PRINCIPAL ---

def generar_reporte_definitivo():
    print("\n--- 🚀 GENERADOR DANONE FINAL: TODO INCLUIDO ---")

    # Inputs
    marca = input("1. Marca: ")
    mes = input("2. Mes: ")
    sprint_num = input("3. Sprint: ")

    nombre_excel = f"Sprint_{sprint_num}_Data_{marca}_Final.xlsx"
    nombre_slides = f"Sprint_{sprint_num}_Presentacion_{marca}_Final.pptx"
    ruta_excel = os.path.join(DIRECTORIO_SALIDA, nombre_excel)
    ruta_slides = os.path.join(DIRECTORIO_SALIDA, nombre_slides)

    # --- A. CARGA Y LIMPIEZA DE DATOS ---
    if not os.path.exists(RUTA_ENTRADA):
        print("❌ Error: No se encuentra el CSV de datos.")
        return

    try:
        df = pd.read_csv(RUTA_ENTRADA, encoding='utf-8')
    except:
        df = pd.read_csv(RUTA_ENTRADA, encoding='latin-1')

    # Convertir a numérico
    cols_num = ['Keyword quality score', 'Clicks', 'Cost', 'Impressions', 'Avg. CPC']
    for c in cols_num:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors='coerce').fillna(0)

    if 'Keyword quality score' in df.columns:
        df['Keyword quality score'] = df['Keyword quality score'].astype(int)

    # Cálculos KPIs Generales
    total_impr = df['Impressions'].sum()
    total_clics = df['Clicks'].sum()
    total_cost = df['Cost'].sum()
    avg_cpc = total_cost / total_clics if total_clics > 0 else 0
    avg_ctr = (total_clics / total_impr * 100) if total_impr > 0 else 0

    # --- B. GRÁFICOS (Para PPT) ---
    print("🎨 Generando gráficos PNG...")
    plt.style.use('ggplot')

    # 1. Pie Chart
    counts = df.groupby('Keyword quality score').size()
    counts_plot = counts[counts > 0]
    plt.figure(figsize=(7, 7))
    colors = plt.cm.Blues(np.linspace(0.4, 0.9, len(counts_plot)))
    plt.pie(counts_plot, labels=counts_plot.index, autopct='%1.1f%%', colors=colors)
    plt.title(f'Share de Calidad - {marca}', color='#004DA5', fontweight='bold')
    plt.savefig('chart_pie.png', bbox_inches='tight')
    plt.close()

    # 2. Bar Chart
    df_eff = df.groupby('Keyword quality score')[['Clicks', 'Cost']].sum()
    df_eff['Avg_CPC'] = (df_eff['Cost'] / df_eff['Clicks']).fillna(0)

    plt.figure(figsize=(9, 5))
    scores_x = np.arange(1, 11)
    cpc_y = [df_eff.loc[s, 'Avg_CPC'] if s in df_eff.index else 0 for s in scores_x]
    plt.bar(scores_x, cpc_y, color='#004DA5')
    plt.title('Eficiencia de Costos (CPC)', color='#004DA5', fontweight='bold')
    plt.xlabel('Quality Score')
    plt.savefig('chart_bar.png', bbox_inches='tight')
    plt.close()

    # --- C. POWERPOINT (Modificación Template) ---
    print("slides Editando Template PPTX...")
    if not os.path.exists(TEMPLATE_PATH):
        print("⚠️ Template no encontrado. Creando uno básico.")
        prs = Presentation()
    else:
        prs = Presentation(TEMPLATE_PATH)

    # 1. Slide 4 (Index 3): RACIONAL ESTRATÉGICO
    # Reemplazamos los #### con texto real usando un cuadro superpuesto limpio
    try:
        slide = prs.slides[3]
        left = Inches(0.5); top = Inches(1.8); width = Inches(9); height = Inches(4.5)
        tx_box = slide.shapes.add_textbox(left, top, width, height)
        tf = tx_box.text_frame
        tf.word_wrap = True

        lines = [
            f"Resultados del Mes: {mes}",
            f"• Impresiones Totales: {total_impr:,.0f}",
            f"• Clics Totales: {total_clics:,.0f}",
            f"• CTR Promedio: {avg_ctr:.2f}%",
            f"• CPC Promedio: ${avg_cpc:.2f}",
            f"• Inversión Total: ${total_cost:,.2f}"
        ]

        for line in lines:
            p = tf.add_paragraph()
            p.text = line
            p.font.name = 'Arial'
            p.font.size = Pt(22)
            p.font.color.rgb = RGBColor(80, 80, 80)
            p.space_after = Pt(14)

    except IndexError: pass

    # 2. Slide 5 (Index 4): PERSPECTIVA DE CALIDAD
    try:
        slide = prs.slides[4]
        limpiar_area_grafico(slide)
        poner_grafico(slide, 'chart_pie.png')
    except IndexError: pass

    # 3. Slide 6 (Index 5): PERSPECTIVA DE PRECIO
    try:
        slide = prs.slides[5]
        limpiar_area_grafico(slide)
        poner_grafico(slide, 'chart_bar.png')
    except IndexError: pass

    prs.save(ruta_slides)

    # --- D. EXCEL MAESTRO (CON TODA LA LÓGICA) ---
    print("📊 Generando Excel Maestro...")
    writer = pd.ExcelWriter(ruta_excel, engine='xlsxwriter')
    wb = writer.book

    # ESTILOS
    # Cabecera: Negro con letras blancas
    header_fmt = wb.add_format({
        'bold': True, 'bg_color': '#000000', 'font_color': '#FFFFFF',
        'border': 1, 'align': 'center', 'valign': 'vcenter'
    })
    # Moneda: 2 decimales
    currency_fmt = wb.add_format({'num_format': '$#,##0.00'})

    # >>> PESTAÑA 1: RACIONAL ESTRATÉGICO <<<
    ws_racional = wb.add_worksheet('Racional estrategico')
    writer.sheets['Racional estrategico'] = ws_racional

    ws_racional.write(0, 0, f"AUDITORÍA: {marca}", wb.add_format({'bold': True, 'size': 14}))

    # Tabla Resumen
    data_kpi = [
        ["Métrica", "Valor"],
        ["Impresiones", total_impr],
        ["Clics", total_clics],
        ["Inversión", total_cost],
        ["CPC Medio", avg_cpc]
    ]

    row = 2
    for item in data_kpi:
        ws_racional.write(row, 0, item[0], wb.add_format({'bold': True}))
        # Aplicar formato moneda si corresponde
        fmt = currency_fmt if row >= 4 else None
        ws_racional.write(row, 1, item[1], fmt)
        row += 1

    # Tabla para el gráfico
    row += 2
    ws_racional.write(row, 0, "Quality Score", header_fmt)
    ws_racional.write(row, 1, "Keywords", header_fmt)
    row += 1

    start_chart_row = row
    counts_dict = counts.to_dict()
    for q in range(1, 11):
        ws_racional.write(row, 0, q)
        ws_racional.write(row, 1, counts_dict.get(q, 0))
        row += 1
    end_chart_row = row - 1

    # Gráfico de Pastel Excel
    chart = wb.add_chart({'type': 'pie'})
    chart.add_series({
        'name': 'Distribución Calidad',
        'categories': ['Racional estrategico', start_chart_row, 0, end_chart_row, 0],
        'values':     ['Racional estrategico', start_chart_row, 1, end_chart_row, 1],
    })
    ws_racional.insert_chart('D2', chart)

    # >>> PESTAÑAS 2..11: QUALITY SCORE DETALLADO <<<
    cols_export = ['Campaign', 'Ad group', 'Search keyword', 'Clicks', 'Impressions', 'CTR', 'Avg. CPC']

    for score in range(1, 11):
        sheet_name = f'Quality {score}'

        # 1. Filtrar
        dft = df[df['Keyword quality score'] == score][cols_export].copy()

        if not dft.empty:
            # 2. Ordenar: CPC Mayor a Menor
            dft = dft.sort_values(by='Avg. CPC', ascending=False)

            # 3. Escribir datos (fila 1 en adelante)
            dft.to_excel(writer, sheet_name=sheet_name, index=False, startrow=1, header=False)

            ws = writer.sheets[sheet_name]

            # 4. Escribir Cabecera Personalizada (Negro/Blanco)
            for col_idx, col_name in enumerate(cols_export):
                ws.write(0, col_idx, col_name, header_fmt)

            # 5. Ajustar anchos
            ws.set_column('A:B', 25)
            ws.set_column('C:C', 40)
            ws.set_column('G:G', 12, currency_fmt) # Formato Moneda Columna G

            # 6. >>> DEGRADADO (HEATMAP) <<<
            # Columna G es índice 6.
            # Filas: desde 1 hasta len(dft)
            ws.conditional_format(1, 6, len(dft), 6, {
                'type': '3_color_scale',
                'min_color': '#FFFFFF', # Blanco
                'mid_color': '#FFF2CC', # Amarillo claro
                'max_color': '#FFD700'  # Dorado fuerte
            })

    writer.close()
    print("\n✅ ¡LISTO! Reporte generado correctamente.")
    print(f"📂 Excel: {nombre_excel}")
    print(f"📂 Slides: {nombre_slides}")

# Ejecutar
generar_reporte_definitivo()


--- 🚀 GENERADOR DANONE FINAL: TODO INCLUIDO ---
1. Marca: Danone
2. Mes: enero
3. Sprint: 433
🎨 Generando gráficos PNG...
slides Editando Template PPTX...
📊 Generando Excel Maestro...

✅ ¡LISTO! Reporte generado correctamente.
📂 Excel: Sprint_433_Data_Danone_Final.xlsx
📂 Slides: Sprint_433_Presentacion_Danone_Final.pptx


## **Danonino**

In [10]:
# --- 1. CONFIGURACIÓN ---
RUTA_ENTRADA = '/content/drive/MyDrive/Danone_SEO/Danonino/holistic-danonino-1.csv'
DIRECTORIO_SALIDA = '/content/drive/MyDrive/Danone_SEO/Danonino/'
TEMPLATE_PATH = '/content/drive/MyDrive/Danone 2026/7. Holistic /Assets/SEO_Danone-Holistic-template.pptx'

# Colores
DANONE_BLUE = RGBColor(0, 77, 165)
DANONE_WHITE = RGBColor(255, 255, 255)

# --- 2. FUNCIONES DE APOYO PPT ---

def limpiar_area_grafico(slide):
    """Intenta limpiar el centro de la slide para poner el gráfico nuevo"""
    for shape in list(slide.shapes):
        # Si es un placeholder de imagen o un cuadro vacío grande, lo borramos
        if shape.is_placeholder and shape.placeholder_format.type == 18: # PICTURE
            sp = shape._element
            sp.getparent().remove(sp)

def poner_grafico(slide, ruta_img):
    """Inserta la imagen del gráfico centrada"""
    # Ajuste manual para que quede bien en el template Danone
    left = Inches(1)
    top = Inches(2.2)
    height = Inches(4.2)
    slide.shapes.add_picture(ruta_img, left, top, height=height)

# --- 3. PROCESO PRINCIPAL ---

def generar_reporte_definitivo():
    print("\n--- 🚀 GENERADOR DANONE FINAL: TODO INCLUIDO ---")

    # Inputs
    marca = input("1. Marca: ")
    mes = input("2. Mes: ")
    sprint_num = input("3. Sprint: ")

    nombre_excel = f"Sprint_{sprint_num}_Data_{marca}_Final.xlsx"
    nombre_slides = f"Sprint_{sprint_num}_Presentacion_{marca}_Final.pptx"
    ruta_excel = os.path.join(DIRECTORIO_SALIDA, nombre_excel)
    ruta_slides = os.path.join(DIRECTORIO_SALIDA, nombre_slides)

    # --- A. CARGA Y LIMPIEZA DE DATOS ---
    if not os.path.exists(RUTA_ENTRADA):
        print("❌ Error: No se encuentra el CSV de datos.")
        return

    try:
        df = pd.read_csv(RUTA_ENTRADA, encoding='utf-8')
    except:
        df = pd.read_csv(RUTA_ENTRADA, encoding='latin-1')

    # Convertir a numérico
    cols_num = ['Keyword quality score', 'Clicks', 'Cost', 'Impressions', 'Avg. CPC']
    for c in cols_num:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors='coerce').fillna(0)

    if 'Keyword quality score' in df.columns:
        df['Keyword quality score'] = df['Keyword quality score'].astype(int)

    # Cálculos KPIs Generales
    total_impr = df['Impressions'].sum()
    total_clics = df['Clicks'].sum()
    total_cost = df['Cost'].sum()
    avg_cpc = total_cost / total_clics if total_clics > 0 else 0
    avg_ctr = (total_clics / total_impr * 100) if total_impr > 0 else 0

    # --- B. GRÁFICOS (Para PPT) ---
    print("🎨 Generando gráficos PNG...")
    plt.style.use('ggplot')

    # 1. Pie Chart
    counts = df.groupby('Keyword quality score').size()
    counts_plot = counts[counts > 0]
    plt.figure(figsize=(7, 7))
    colors = plt.cm.Blues(np.linspace(0.4, 0.9, len(counts_plot)))
    plt.pie(counts_plot, labels=counts_plot.index, autopct='%1.1f%%', colors=colors)
    plt.title(f'Share de Calidad - {marca}', color='#004DA5', fontweight='bold')
    plt.savefig('chart_pie.png', bbox_inches='tight')
    plt.close()

    # 2. Bar Chart
    df_eff = df.groupby('Keyword quality score')[['Clicks', 'Cost']].sum()
    df_eff['Avg_CPC'] = (df_eff['Cost'] / df_eff['Clicks']).fillna(0)

    plt.figure(figsize=(9, 5))
    scores_x = np.arange(1, 11)
    cpc_y = [df_eff.loc[s, 'Avg_CPC'] if s in df_eff.index else 0 for s in scores_x]
    plt.bar(scores_x, cpc_y, color='#004DA5')
    plt.title('Eficiencia de Costos (CPC)', color='#004DA5', fontweight='bold')
    plt.xlabel('Quality Score')
    plt.savefig('chart_bar.png', bbox_inches='tight')
    plt.close()

    # --- C. POWERPOINT (Modificación Template) ---
    print("slides Editando Template PPTX...")
    if not os.path.exists(TEMPLATE_PATH):
        print("⚠️ Template no encontrado. Creando uno básico.")
        prs = Presentation()
    else:
        prs = Presentation(TEMPLATE_PATH)

    # 1. Slide 4 (Index 3): RACIONAL ESTRATÉGICO
    # Reemplazamos los #### con texto real usando un cuadro superpuesto limpio
    try:
        slide = prs.slides[3]
        left = Inches(0.5); top = Inches(1.8); width = Inches(9); height = Inches(4.5)
        tx_box = slide.shapes.add_textbox(left, top, width, height)
        tf = tx_box.text_frame
        tf.word_wrap = True

        lines = [
            f"Resultados del Mes: {mes}",
            f"• Impresiones Totales: {total_impr:,.0f}",
            f"• Clics Totales: {total_clics:,.0f}",
            f"• CTR Promedio: {avg_ctr:.2f}%",
            f"• CPC Promedio: ${avg_cpc:.2f}",
            f"• Inversión Total: ${total_cost:,.2f}"
        ]

        for line in lines:
            p = tf.add_paragraph()
            p.text = line
            p.font.name = 'Arial'
            p.font.size = Pt(22)
            p.font.color.rgb = RGBColor(80, 80, 80)
            p.space_after = Pt(14)

    except IndexError: pass

    # 2. Slide 5 (Index 4): PERSPECTIVA DE CALIDAD
    try:
        slide = prs.slides[4]
        limpiar_area_grafico(slide)
        poner_grafico(slide, 'chart_pie.png')
    except IndexError: pass

    # 3. Slide 6 (Index 5): PERSPECTIVA DE PRECIO
    try:
        slide = prs.slides[5]
        limpiar_area_grafico(slide)
        poner_grafico(slide, 'chart_bar.png')
    except IndexError: pass

    prs.save(ruta_slides)

    # --- D. EXCEL MAESTRO (CON TODA LA LÓGICA) ---
    print("📊 Generando Excel Maestro...")
    writer = pd.ExcelWriter(ruta_excel, engine='xlsxwriter')
    wb = writer.book

    # ESTILOS
    # Cabecera: Negro con letras blancas
    header_fmt = wb.add_format({
        'bold': True, 'bg_color': '#000000', 'font_color': '#FFFFFF',
        'border': 1, 'align': 'center', 'valign': 'vcenter'
    })
    # Moneda: 2 decimales
    currency_fmt = wb.add_format({'num_format': '$#,##0.00'})

    # >>> PESTAÑA 1: RACIONAL ESTRATÉGICO <<<
    ws_racional = wb.add_worksheet('Racional estrategico')
    writer.sheets['Racional estrategico'] = ws_racional

    ws_racional.write(0, 0, f"AUDITORÍA: {marca}", wb.add_format({'bold': True, 'size': 14}))

    # Tabla Resumen
    data_kpi = [
        ["Métrica", "Valor"],
        ["Impresiones", total_impr],
        ["Clics", total_clics],
        ["Inversión", total_cost],
        ["CPC Medio", avg_cpc]
    ]

    row = 2
    for item in data_kpi:
        ws_racional.write(row, 0, item[0], wb.add_format({'bold': True}))
        # Aplicar formato moneda si corresponde
        fmt = currency_fmt if row >= 4 else None
        ws_racional.write(row, 1, item[1], fmt)
        row += 1

    # Tabla para el gráfico
    row += 2
    ws_racional.write(row, 0, "Quality Score", header_fmt)
    ws_racional.write(row, 1, "Keywords", header_fmt)
    row += 1

    start_chart_row = row
    counts_dict = counts.to_dict()
    for q in range(1, 11):
        ws_racional.write(row, 0, q)
        ws_racional.write(row, 1, counts_dict.get(q, 0))
        row += 1
    end_chart_row = row - 1

    # Gráfico de Pastel Excel
    chart = wb.add_chart({'type': 'pie'})
    chart.add_series({
        'name': 'Distribución Calidad',
        'categories': ['Racional estrategico', start_chart_row, 0, end_chart_row, 0],
        'values':     ['Racional estrategico', start_chart_row, 1, end_chart_row, 1],
    })
    ws_racional.insert_chart('D2', chart)

    # >>> PESTAÑAS 2..11: QUALITY SCORE DETALLADO <<<
    cols_export = ['Campaign', 'Ad group', 'Search keyword', 'Clicks', 'Impressions', 'CTR', 'Avg. CPC']

    for score in range(1, 11):
        sheet_name = f'Quality {score}'

        # 1. Filtrar
        dft = df[df['Keyword quality score'] == score][cols_export].copy()

        if not dft.empty:
            # 2. Ordenar: CPC Mayor a Menor
            dft = dft.sort_values(by='Avg. CPC', ascending=False)

            # 3. Escribir datos (fila 1 en adelante)
            dft.to_excel(writer, sheet_name=sheet_name, index=False, startrow=1, header=False)

            ws = writer.sheets[sheet_name]

            # 4. Escribir Cabecera Personalizada (Negro/Blanco)
            for col_idx, col_name in enumerate(cols_export):
                ws.write(0, col_idx, col_name, header_fmt)

            # 5. Ajustar anchos
            ws.set_column('A:B', 25)
            ws.set_column('C:C', 40)
            ws.set_column('G:G', 12, currency_fmt) # Formato Moneda Columna G

            # 6. >>> DEGRADADO (HEATMAP) <<<
            # Columna G es índice 6.
            # Filas: desde 1 hasta len(dft)
            ws.conditional_format(1, 6, len(dft), 6, {
                'type': '3_color_scale',
                'min_color': '#FFFFFF', # Blanco
                'mid_color': '#FFF2CC', # Amarillo claro
                'max_color': '#FFD700'  # Dorado fuerte
            })

    writer.close()
    print("\n✅ ¡LISTO! Reporte generado correctamente.")
    print(f"📂 Excel: {nombre_excel}")
    print(f"📂 Slides: {nombre_slides}")

# Ejecutar
generar_reporte_definitivo()


--- 🚀 GENERADOR DANONE FINAL: TODO INCLUIDO ---
1. Marca: Danonino
2. Mes: Enero
3. Sprint: 2
🎨 Generando gráficos PNG...
slides Editando Template PPTX...
📊 Generando Excel Maestro...

✅ ¡LISTO! Reporte generado correctamente.
📂 Excel: Sprint_2_Data_Danonino_Final.xlsx
📂 Slides: Sprint_2_Presentacion_Danonino_Final.pptx


## **SILK**

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.dml.color import RGBColor
import os

# --- 1. CONFIGURACIÓN ---
RUTA_ENTRADA = '/content/drive/MyDrive/Danone_SEO/Silk/Silk-data-enero.csv'
DIRECTORIO_SALIDA = '/content/drive/MyDrive/Danone_SEO/Silk/'
TEMPLATE_PATH = '/content/drive/MyDrive/Danone 2026/7. Holistic /Assets/SEO_Danone-Holistic-template.pptx'

# Colores
DANONE_BLUE = RGBColor(0, 77, 165)
DANONE_WHITE = RGBColor(255, 255, 255)

# --- 2. FUNCIONES DE APOYO PPT ---

def limpiar_area_grafico(slide):
    """Intenta limpiar el centro de la slide para poner el gráfico nuevo"""
    for shape in list(slide.shapes):
        # Si es un placeholder de imagen o un cuadro vacío grande, lo borramos
        if shape.is_placeholder and shape.placeholder_format.type == 18: # PICTURE
            sp = shape._element
            sp.getparent().remove(sp)

def poner_grafico(slide, ruta_img):
    """Inserta la imagen del gráfico centrada"""
    # Ajuste manual para que quede bien en el template Danone
    left = Inches(1)
    top = Inches(2.2)
    height = Inches(4.2)
    slide.shapes.add_picture(ruta_img, left, top, height=height)

# --- 3. PROCESO PRINCIPAL ---

def generar_reporte_definitivo():
    print("\n--- 🚀 GENERADOR DANONE FINAL: TODO INCLUIDO ---")

    # Inputs
    marca = input("1. Marca: ")
    mes = input("2. Mes: ")
    sprint_num = input("3. Sprint: ")

    nombre_excel = f"Sprint_{sprint_num}_Data_{marca}_Final.xlsx"
    nombre_slides = f"Sprint_{sprint_num}_Presentacion_{marca}_Final.pptx"
    ruta_excel = os.path.join(DIRECTORIO_SALIDA, nombre_excel)
    ruta_slides = os.path.join(DIRECTORIO_SALIDA, nombre_slides)

    # --- A. CARGA Y LIMPIEZA DE DATOS ---
    if not os.path.exists(RUTA_ENTRADA):
        print("❌ Error: No se encuentra el CSV de datos.")
        return

    try:
        df = pd.read_csv(RUTA_ENTRADA, encoding='utf-8')
    except:
        df = pd.read_csv(RUTA_ENTRADA, encoding='latin-1')

    # Convertir a numérico
    cols_num = ['Keyword quality score', 'Clicks', 'Cost', 'Impressions', 'Avg. CPC']
    for c in cols_num:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors='coerce').fillna(0)

    if 'Keyword quality score' in df.columns:
        df['Keyword quality score'] = df['Keyword quality score'].astype(int)

    # Cálculos KPIs Generales
    total_impr = df['Impressions'].sum()
    total_clics = df['Clicks'].sum()
    total_cost = df['Cost'].sum()
    avg_cpc = total_cost / total_clics if total_clics > 0 else 0
    avg_ctr = (total_clics / total_impr * 100) if total_impr > 0 else 0

    # --- B. GRÁFICOS (Para PPT) ---
    print("🎨 Generando gráficos PNG...")
    plt.style.use('ggplot')

    # 1. Pie Chart
    counts = df.groupby('Keyword quality score').size()
    counts_plot = counts[counts > 0]
    plt.figure(figsize=(7, 7))
    colors = plt.cm.Blues(np.linspace(0.4, 0.9, len(counts_plot)))
    plt.pie(counts_plot, labels=counts_plot.index, autopct='%1.1f%%', colors=colors)
    plt.title(f'Share de Calidad - {marca}', color='#004DA5', fontweight='bold')
    plt.savefig('chart_pie.png', bbox_inches='tight')
    plt.close()

    # 2. Bar Chart
    df_eff = df.groupby('Keyword quality score')[['Clicks', 'Cost']].sum()
    df_eff['Avg_CPC'] = (df_eff['Cost'] / df_eff['Clicks']).fillna(0)

    plt.figure(figsize=(9, 5))
    scores_x = np.arange(1, 11)
    cpc_y = [df_eff.loc[s, 'Avg_CPC'] if s in df_eff.index else 0 for s in scores_x]
    plt.bar(scores_x, cpc_y, color='#004DA5')
    plt.title('Eficiencia de Costos (CPC)', color='#004DA5', fontweight='bold')
    plt.xlabel('Quality Score')
    plt.savefig('chart_bar.png', bbox_inches='tight')
    plt.close()

    # --- C. POWERPOINT (Modificación Template) ---
    print("slides Editando Template PPTX...")
    if not os.path.exists(TEMPLATE_PATH):
        print("⚠️ Template no encontrado. Creando uno básico.")
        prs = Presentation()
    else:
        prs = Presentation(TEMPLATE_PATH)

    # 1. Slide 4 (Index 3): RACIONAL ESTRATÉGICO
    # Reemplazamos los #### con texto real usando un cuadro superpuesto limpio
    try:
        slide = prs.slides[3]
        left = Inches(0.5); top = Inches(1.8); width = Inches(9); height = Inches(4.5)
        tx_box = slide.shapes.add_textbox(left, top, width, height)
        tf = tx_box.text_frame
        tf.word_wrap = True

        lines = [
            f"Resultados del Mes: {mes}",
            f"• Impresiones Totales: {total_impr:,.0f}",
            f"• Clics Totales: {total_clics:,.0f}",
            f"• CTR Promedio: {avg_ctr:.2f}%",
            f"• CPC Promedio: ${avg_cpc:.2f}",
            f"• Inversión Total: ${total_cost:,.2f}"
        ]

        for line in lines:
            p = tf.add_paragraph()
            p.text = line
            p.font.name = 'Arial'
            p.font.size = Pt(22)
            p.font.color.rgb = RGBColor(80, 80, 80)
            p.space_after = Pt(14)

    except IndexError: pass

    # 2. Slide 5 (Index 4): PERSPECTIVA DE CALIDAD
    try:
        slide = prs.slides[4]
        limpiar_area_grafico(slide)
        poner_grafico(slide, 'chart_pie.png')
    except IndexError: pass

    # 3. Slide 6 (Index 5): PERSPECTIVA DE PRECIO
    try:
        slide = prs.slides[5]
        limpiar_area_grafico(slide)
        poner_grafico(slide, 'chart_bar.png')
    except IndexError: pass

    prs.save(ruta_slides)

    # --- D. EXCEL MAESTRO (CON TODA LA LÓGICA) ---
    print("📊 Generando Excel Maestro...")
    writer = pd.ExcelWriter(ruta_excel, engine='xlsxwriter')
    wb = writer.book

    # ESTILOS
    # Cabecera: Negro con letras blancas
    header_fmt = wb.add_format({
        'bold': True, 'bg_color': '#000000', 'font_color': '#FFFFFF',
        'border': 1, 'align': 'center', 'valign': 'vcenter'
    })
    # Moneda: 2 decimales
    currency_fmt = wb.add_format({'num_format': '$#,##0.00'})

    # >>> PESTAÑA 1: RACIONAL ESTRATÉGICO <<<
    ws_racional = wb.add_worksheet('Racional estrategico')
    writer.sheets['Racional estrategico'] = ws_racional

    ws_racional.write(0, 0, f"AUDITORÍA: {marca}", wb.add_format({'bold': True, 'size': 14}))

    # Tabla Resumen
    data_kpi = [
        ["Métrica", "Valor"],
        ["Impresiones", total_impr],
        ["Clics", total_clics],
        ["Inversión", total_cost],
        ["CPC Medio", avg_cpc]
    ]

    row = 2
    for item in data_kpi:
        ws_racional.write(row, 0, item[0], wb.add_format({'bold': True}))
        # Aplicar formato moneda si corresponde
        fmt = currency_fmt if row >= 4 else None
        ws_racional.write(row, 1, item[1], fmt)
        row += 1

    # Tabla para el gráfico
    row += 2
    ws_racional.write(row, 0, "Quality Score", header_fmt)
    ws_racional.write(row, 1, "Keywords", header_fmt)
    row += 1

    start_chart_row = row
    counts_dict = counts.to_dict()
    for q in range(1, 11):
        ws_racional.write(row, 0, q)
        ws_racional.write(row, 1, counts_dict.get(q, 0))
        row += 1
    end_chart_row = row - 1

    # Gráfico de Pastel Excel
    chart = wb.add_chart({'type': 'pie'})
    chart.add_series({
        'name': 'Distribución Calidad',
        'categories': ['Racional estrategico', start_chart_row, 0, end_chart_row, 0],
        'values':     ['Racional estrategico', start_chart_row, 1, end_chart_row, 1],
    })
    ws_racional.insert_chart('D2', chart)

    # >>> PESTAÑAS 2..11: QUALITY SCORE DETALLADO <<<
    cols_export = ['Campaign', 'Ad group', 'Search keyword', 'Clicks', 'Impressions', 'CTR', 'Avg. CPC']

    for score in range(1, 11):
        sheet_name = f'Quality {score}'

        # 1. Filtrar
        dft = df[df['Keyword quality score'] == score][cols_export].copy()

        if not dft.empty:
            # 2. Ordenar: CPC Mayor a Menor
            dft = dft.sort_values(by='Avg. CPC', ascending=False)

            # 3. Escribir datos (fila 1 en adelante)
            dft.to_excel(writer, sheet_name=sheet_name, index=False, startrow=1, header=False)

            ws = writer.sheets[sheet_name]

            # 4. Escribir Cabecera Personalizada (Negro/Blanco)
            for col_idx, col_name in enumerate(cols_export):
                ws.write(0, col_idx, col_name, header_fmt)

            # 5. Ajustar anchos
            ws.set_column('A:B', 25)
            ws.set_column('C:C', 40)
            ws.set_column('G:G', 12, currency_fmt) # Formato Moneda Columna G

            # 6. >>> DEGRADADO (HEATMAP) <<<
            # Columna G es índice 6.
            # Filas: desde 1 hasta len(dft)
            ws.conditional_format(1, 6, len(dft), 6, {
                'type': '3_color_scale',
                'min_color': '#FFFFFF', # Blanco
                'mid_color': '#FFF2CC', # Amarillo claro
                'max_color': '#FFD700'  # Dorado fuerte
            })

    writer.close()
    print("\n✅ ¡LISTO! Reporte generado correctamente.")
    print(f"📂 Excel: {nombre_excel}")
    print(f"📂 Slides: {nombre_slides}")

# Ejecutar
generar_reporte_definitivo()


--- 🚀 GENERADOR DANONE FINAL: TODO INCLUIDO ---
1. Marca: Silk
2. Mes: Enero
3. Sprint: 3
🎨 Generando gráficos PNG...
slides Editando Template PPTX...
📊 Generando Excel Maestro...

✅ ¡LISTO! Reporte generado correctamente.
📂 Excel: Sprint_3_Data_Silk_Final.xlsx
📂 Slides: Sprint_3_Presentacion_Silk_Final.pptx


## **OIKOS**